In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from sklearn import preprocessing
import re
import csv

In [ ]:
data = pd.read_csv('zomato_restaurants_in_India.csv')

In [ ]:
data.shape

(211944, 26)

In [ ]:
data.head(3)

,res_id,name,establishment,url,address,city,city_id,locality,latitude,longitude,zipcode,country_id,locality_verbose,cuisines,timings,average_cost_for_two,price_range,currency,highlights,aggregate_rating,rating_text,votes,photo_count,opentable_support,delivery,takeaway
0,3400299,Bikanervala,['Quick Bites'],https://www.zomato.com/agra/bikanervala-khanda...,"Kalyani Point, Near Tulsi Cinema, Bypass Road,...",Agra,34,Khandari,27.211450,78.002381,NaN,1,"Khandari, Agra","North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),700,2,Rs.,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,Very Good,814,154,0.0,-1,-1
1,3400005,Mama Chicken Mama Franky House,['Quick Bites'],https://www.zomato.com/agra/mama-chicken-mama-...,"Main Market, Sadar Bazaar, Agra Cantt, Agra",Agra,34,Agra Cantt,27.160569,78.011583,282001.0,1,"Agra Cantt, Agra","North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",600,2,Rs.,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,Very Good,1203,161,0.0,-1,-1
2,3401013,Bhagat Halwai,['Quick Bites'],https://www.zomato.com/agra/bhagat-halwai-2-sh...,"62/1, Near Easy Day, West Shivaji Nagar, Goalp...",Agra,34,Shahganj,27.182938,77.979684,282010.0,1,"Shahganj, Agra","Fast Food, Mithai",9:30 AM to 11 PM,300,1,Rs.,"['No Alcohol Available', 'Dinner', 'Takeaway A...",4.2,Very Good,801,107,0.0,1,-1


In [ ]:
drop_cols = ['res_id', 'url', 'address', 'city', 'locality', 'latitude', 'longitude', 'zipcode', 'country_id', 'currency', 'rating_text', 'votes', 'photo_count', 'opentable_support', 'takeaway']
data = data.drop(columns=drop_cols)

In [ ]:
data.head(3)

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,price_range,highlights,aggregate_rating,delivery
0,Bikanervala,['Quick Bites'],34,"Khandari, Agra","North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),700,2,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1
1,Mama Chicken Mama Franky House,['Quick Bites'],34,"Agra Cantt, Agra","North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",600,2,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1
2,Bhagat Halwai,['Quick Bites'],34,"Shahganj, Agra","Fast Food, Mithai",9:30 AM to 11 PM,300,1,"['No Alcohol Available', 'Dinner', 'Takeaway A...",4.2,1


## Data info

In [ ]:
data.loc[data['average_cost_for_two'].idxmax()]

name                        Ocean - The Private Dining Room - Sahara Star
establishment                                             ['Fine Dining']
city_id                                                                 3
locality_verbose                     Sahara Star, Vile Parle East, Mumbai
cuisines                      Asian, Seafood, North Indian, Mediterranean
timings                    12 Noon to 3 PM, 7 PM to 12 Midnight (Mon-Sun)
average_cost_for_two                                                30000
price_range                                                             4
highlights              ['Dinner', 'Serves Alcohol', 'Debit Card', 'Ca...
aggregate_rating                                                      3.6
delivery                                                               -1
Name: 136240, dtype: object

In [ ]:
data.describe()

,city_id,average_cost_for_two,price_range,aggregate_rating,delivery
count,211944.000000,211944.000000,211944.000000,211944.000000,211944.000000
mean,4746.785434,595.812229,1.882535,3.395937,-0.255907
std,5568.766386,606.239363,0.892989,1.283642,0.964172
min,1.000000,0.000000,1.000000,0.000000,-1.000000
25%,11.000000,250.000000,1.000000,3.300000,-1.000000
50%,34.000000,400.000000,2.000000,3.800000,-1.000000
75%,11306.000000,700.000000,2.000000,4.100000,1.000000
max,11354.000000,30000.000000,4.000000,4.900000,1.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211944 entries, 0 to 211943
Data columns (total 11 columns):
name                    211944 non-null object
establishment           211944 non-null object
city_id                 211944 non-null int64
locality_verbose        211944 non-null object
cuisines                210553 non-null object
timings                 208070 non-null object
average_cost_for_two    211944 non-null int64
price_range             211944 non-null int64
highlights              211944 non-null object
aggregate_rating        211944 non-null float64
delivery                211944 non-null int64
dtypes: float64(1), int64(4), object(6)
memory usage: 17.8+ MB


In [ ]:
df = data.dropna(axis=0, how='any')

In [ ]:
df.head(5)

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,price_range,highlights,aggregate_rating,delivery
0,Bikanervala,['Quick Bites'],34,"Khandari, Agra","North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),700,2,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1
1,Mama Chicken Mama Franky House,['Quick Bites'],34,"Agra Cantt, Agra","North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",600,2,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1
2,Bhagat Halwai,['Quick Bites'],34,"Shahganj, Agra","Fast Food, Mithai",9:30 AM to 11 PM,300,1,"['No Alcohol Available', 'Dinner', 'Takeaway A...",4.2,1
3,Bhagat Halwai,['Quick Bites'],34,"Civil Lines, Agra","Desserts, Bakery, Fast Food, South Indian",8am – 11pm (Mon-Sun),300,1,"['Takeaway Available', 'Credit Card', 'Lunch',...",4.3,1
4,The Salt Cafe Kitchen & Bar,['Casual Dining'],34,"Tajganj, Agra","North Indian, Continental, Italian",11:30 AM to 11:30 PM,1000,3,"['Lunch', 'Serves Alcohol', 'Cash', 'Credit Ca...",4.9,1


# Pre-processing

## Label encoding Name & Establishment

In [ ]:
label_encoder = preprocessing.LabelEncoder() 

In [ ]:
df['name'] = label_encoder.fit_transform(df['name'])

C:\Users\saini\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
est_list = []
for est in df.establishment.values:
    est = re.sub('[^A-Za-z0-9 ]+', '', est)
    if est not in est_list:
        est_list.append(est)
with open('establishment.txt', 'w') as f:
    f.write(',\n'.join(sorted(est_list)))

In [ ]:
df['establishment'] = label_encoder.fit_transform(df['establishment'])

C:\Users\saini\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
locality_list = []
for locality in df.locality_verbose.values:
    locality = re.sub('[^A-Za-z0-9, ]+', '', locality)
    if locality not in locality_list:
        locality_list.append(locality)
with open('locality.txt', 'w') as f:
    f.write(';\n'.join(sorted(locality_list)))

In [ ]:
df['locality_verbose'] = label_encoder.fit_transform(df['locality_verbose'])

C:\Users\saini\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.head()

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,price_range,highlights,aggregate_rating,delivery
0,4476,23,34,1764,"North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),700,2,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1
1,21211,23,34,74,"North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",600,2,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1
2,4100,23,34,3149,"Fast Food, Mithai",9:30 AM to 11 PM,300,1,"['No Alcohol Available', 'Dinner', 'Takeaway A...",4.2,1
3,4100,23,34,600,"Desserts, Bakery, Fast Food, South Indian",8am – 11pm (Mon-Sun),300,1,"['Takeaway Available', 'Credit Card', 'Lunch',...",4.3,1
4,36830,6,34,3386,"North Indian, Continental, Italian",11:30 AM to 11:30 PM,1000,3,"['Lunch', 'Serves Alcohol', 'Cash', 'Credit Ca...",4.9,1


## Splitting Cuisines & List creation

In [ ]:
cuisines_list = []
for group in df.cuisines.values:
    cuisines = group.split(",")
    for cuisine in cuisines:
        cuisine = cuisine.strip().rstrip()
        if cuisine not in cuisines_list:
            cuisines_list.append(cuisine)

In [ ]:
cuisines_list = sorted(cuisines_list)

In [ ]:
len(cuisines_list)

133

In [ ]:
with open('cuisines.txt', 'w') as f:
    f.write(',\n'.join(cuisines_list))

In [ ]:
def set_cuisines(row):
#     print(row.cuisines.split(","))
    for ele in row.cuisines.split(","):
#         print(ele)
        cuisine_no = cuisines_list.index(ele.strip().rstrip()) + 1
        row['cuisine_'+str(cuisine_no)] = 1
    return row

In [ ]:
for i in range(len(cuisines_list)):
    df['cuisine_'+str(i+1)] = 0
df = df.apply(set_cuisines, axis=1)

C:\Users\saini\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
df.head(2)

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,price_range,highlights,aggregate_rating,delivery,cuisine_1,cuisine_2,cuisine_3,cuisine_4,cuisine_5,cuisine_6,cuisine_7,cuisine_8,cuisine_9,cuisine_10,cuisine_11,cuisine_12,cuisine_13,cuisine_14,cuisine_15,cuisine_16,cuisine_17,cuisine_18,cuisine_19,cuisine_20,cuisine_21,cuisine_22,cuisine_23,cuisine_24,cuisine_25,cuisine_26,cuisine_27,cuisine_28,cuisine_29,cuisine_30,cuisine_31,cuisine_32,cuisine_33,cuisine_34,cuisine_35,cuisine_36,cuisine_37,cuisine_38,cuisine_39,cuisine_40,cuisine_41,cuisine_42,cuisine_43,cuisine_44,cuisine_45,cuisine_46,cuisine_47,cuisine_48,cuisine_49,cuisine_50,cuisine_51,cuisine_52,cuisine_53,cuisine_54,cuisine_55,cuisine_56,cuisine_57,cuisine_58,cuisine_59,cuisine_60,cuisine_61,cuisine_62,cuisine_63,cuisine_64,cuisine_65,cuisine_66,cuisine_67,cuisine_68,cuisine_69,cuisine_70,cuisine_71,cuisine_72,cuisine_73,cuisine_74,cuisine_75,cuisine_76,cuisine_77,cuisine_78,cuisine_79,cuisine_80,cuisine_81,cuisine_82,cuisine_83,cuisine_84,cuisine_85,cuisine_86,cuisine_87,cuisine_88,cuisine_89,cuisine_90,cuisine_91,cuisine_92,cuisine_93,cuisine_94,cuisine_95,cuisine_96,cuisine_97,cuisine_98,cuisine_99,cuisine_100,cuisine_101,cuisine_102,cuisine_103,cuisine_104,cuisine_105,cuisine_106,cuisine_107,cuisine_108,cuisine_109,cuisine_110,cuisine_111,cuisine_112,cuisine_113,cuisine_114,cuisine_115,cuisine_116,cuisine_117,cuisine_118,cuisine_119,cuisine_120,cuisine_121,cuisine_122,cuisine_123,cuisine_124,cuisine_125,cuisine_126,cuisine_127,cuisine_128,cuisine_129,cuisine_130,cuisine_131,cuisine_132,cuisine_133
0,4476,23,34,1764,"North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),700,2,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,21211,23,34,74,"North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",600,2,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Splitting Highlights and list creation

In [ ]:
highlights_list = []
for group in df.highlights.values:
    highlights = group.split(",")
    for highlight in highlights:
        highlight = highlight.strip().rstrip()
        highlight = re.sub('[^A-Za-z0-9 ]+', '', highlight)
        if highlight not in highlights_list:
            highlights_list.append(highlight)

In [ ]:
highlights_list = sorted(highlights_list)

In [ ]:
len(highlights_list)

104

In [ ]:
with open('highlights.txt', 'w') as f:
    f.write(',\n'.join(highlights_list))

In [ ]:
def set_highlights(row):
#     print(row.highlights.split(","))
    for ele in row.highlights.split(","):
#         print(ele)
        ele = re.sub('[^A-Za-z0-9 ]+', '', ele)
        highlight_no = highlights_list.index(ele.strip().rstrip()) + 1
        row['highlight_'+str(highlight_no)] = 1
    return row

In [ ]:
for i in range(len(highlights_list)):
    df['highlight_'+str(i+1)] = 0
    
df = df.apply(set_highlights, axis=1)

In [ ]:
df.head(2)

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,price_range,highlights,aggregate_rating,delivery,cuisine_1,cuisine_2,cuisine_3,cuisine_4,cuisine_5,cuisine_6,cuisine_7,cuisine_8,cuisine_9,cuisine_10,cuisine_11,cuisine_12,cuisine_13,cuisine_14,cuisine_15,cuisine_16,cuisine_17,cuisine_18,cuisine_19,cuisine_20,cuisine_21,cuisine_22,cuisine_23,cuisine_24,cuisine_25,cuisine_26,cuisine_27,cuisine_28,cuisine_29,cuisine_30,cuisine_31,cuisine_32,cuisine_33,cuisine_34,cuisine_35,cuisine_36,cuisine_37,cuisine_38,cuisine_39,cuisine_40,cuisine_41,cuisine_42,cuisine_43,cuisine_44,cuisine_45,cuisine_46,cuisine_47,cuisine_48,cuisine_49,cuisine_50,cuisine_51,cuisine_52,cuisine_53,cuisine_54,cuisine_55,cuisine_56,cuisine_57,cuisine_58,cuisine_59,cuisine_60,cuisine_61,cuisine_62,cuisine_63,cuisine_64,cuisine_65,cuisine_66,cuisine_67,cuisine_68,cuisine_69,cuisine_70,cuisine_71,cuisine_72,cuisine_73,cuisine_74,cuisine_75,cuisine_76,cuisine_77,cuisine_78,cuisine_79,cuisine_80,cuisine_81,cuisine_82,cuisine_83,cuisine_84,cuisine_85,cuisine_86,cuisine_87,cuisine_88,cuisine_89,cuisine_90,cuisine_91,cuisine_92,cuisine_93,cuisine_94,cuisine_95,cuisine_96,cuisine_97,cuisine_98,cuisine_99,cuisine_100,cuisine_101,cuisine_102,cuisine_103,cuisine_104,cuisine_105,cuisine_106,cuisine_107,cuisine_108,cuisine_109,cuisine_110,cuisine_111,cuisine_112,cuisine_113,cuisine_114,cuisine_115,cuisine_116,cuisine_117,cuisine_118,cuisine_119,cuisine_120,cuisine_121,cuisine_122,cuisine_123,cuisine_124,cuisine_125,cuisine_126,cuisine_127,cuisine_128,cuisine_129,cuisine_130,cuisine_131,cuisine_132,cuisine_133,highlight_1,highlight_2,highlight_3,highlight_4,highlight_5,highlight_6,highlight_7,highlight_8,highlight_9,highlight_10,highlight_11,highlight_12,highlight_13,highlight_14,highlight_15,highlight_16,highlight_17,highlight_18,highlight_19,highlight_20,highlight_21,highlight_22,highlight_23,highlight_24,highlight_25,highlight_26,highlight_27,highlight_28,highlight_29,highlight_30,highlight_31,highlight_32,highlight_33,highlight_34,highlight_35,highlight_36,highlight_37,highlight_38,highlight_39,highlight_40,highlight_41,highlight_42,highlight_43,highlight_44,highlight_45,highlight_46,highlight_47,highlight_48,highlight_49,highlight_50,highlight_51,highlight_52,highlight_53,highlight_54,highlight_55,highlight_56,highlight_57,highlight_58,highlight_59,highlight_60,highlight_61,highlight_62,highlight_63,highlight_64,highlight_65,highlight_66,highlight_67,highlight_68,highlight_69,highlight_70,highlight_71,highlight_72,highlight_73,highlight_74,highlight_75,highlight_76,highlight_77,highlight_78,highlight_79,highlight_80,highlight_81,highlight_82,highlight_83,highlight_84,highlight_85,highlight_86,highlight_87,highlight_88,highlight_89,highlight_90,highlight_91,highlight_92,highlight_93,highlight_94,highlight_95,highlight_96,highlight_97,highlight_98,highlight_99,highlight_100,highlight_101,highlight_102,highlight_103,highlight_104
0,4476,23,34,1764,"North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),700,2,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,21211,23,34,74,"North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",600,2,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

## Calculating price per head (& dropping price range)

In [ ]:
df['average_cost_for_two'] = df['average_cost_for_two'] / df['price_range']
df['average_cost_for_two'] = df['average_cost_for_two'].astype(int)

In [ ]:
df.rename(columns={"average_cost_for_two": "avg_cost_per_head"})
df = df.drop(['price_range'], axis=1)

In [ ]:
df.head(2)

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,highlights,aggregate_rating,delivery,cuisine_1,cuisine_2,cuisine_3,cuisine_4,cuisine_5,cuisine_6,cuisine_7,cuisine_8,cuisine_9,cuisine_10,cuisine_11,cuisine_12,cuisine_13,cuisine_14,cuisine_15,cuisine_16,cuisine_17,cuisine_18,cuisine_19,cuisine_20,cuisine_21,cuisine_22,cuisine_23,cuisine_24,cuisine_25,cuisine_26,cuisine_27,cuisine_28,cuisine_29,cuisine_30,cuisine_31,cuisine_32,cuisine_33,cuisine_34,cuisine_35,cuisine_36,cuisine_37,cuisine_38,cuisine_39,cuisine_40,cuisine_41,cuisine_42,cuisine_43,cuisine_44,cuisine_45,cuisine_46,cuisine_47,cuisine_48,cuisine_49,cuisine_50,cuisine_51,cuisine_52,cuisine_53,cuisine_54,cuisine_55,cuisine_56,cuisine_57,cuisine_58,cuisine_59,cuisine_60,cuisine_61,cuisine_62,cuisine_63,cuisine_64,cuisine_65,cuisine_66,cuisine_67,cuisine_68,cuisine_69,cuisine_70,cuisine_71,cuisine_72,cuisine_73,cuisine_74,cuisine_75,cuisine_76,cuisine_77,cuisine_78,cuisine_79,cuisine_80,cuisine_81,cuisine_82,cuisine_83,cuisine_84,cuisine_85,cuisine_86,cuisine_87,cuisine_88,cuisine_89,cuisine_90,cuisine_91,cuisine_92,cuisine_93,cuisine_94,cuisine_95,cuisine_96,cuisine_97,cuisine_98,cuisine_99,cuisine_100,cuisine_101,cuisine_102,cuisine_103,cuisine_104,cuisine_105,cuisine_106,cuisine_107,cuisine_108,cuisine_109,cuisine_110,cuisine_111,cuisine_112,cuisine_113,cuisine_114,cuisine_115,cuisine_116,cuisine_117,cuisine_118,cuisine_119,cuisine_120,cuisine_121,cuisine_122,cuisine_123,cuisine_124,cuisine_125,cuisine_126,cuisine_127,cuisine_128,cuisine_129,cuisine_130,cuisine_131,cuisine_132,cuisine_133,highlight_1,highlight_2,highlight_3,highlight_4,highlight_5,highlight_6,highlight_7,highlight_8,highlight_9,highlight_10,highlight_11,highlight_12,highlight_13,highlight_14,highlight_15,highlight_16,highlight_17,highlight_18,highlight_19,highlight_20,highlight_21,highlight_22,highlight_23,highlight_24,highlight_25,highlight_26,highlight_27,highlight_28,highlight_29,highlight_30,highlight_31,highlight_32,highlight_33,highlight_34,highlight_35,highlight_36,highlight_37,highlight_38,highlight_39,highlight_40,highlight_41,highlight_42,highlight_43,highlight_44,highlight_45,highlight_46,highlight_47,highlight_48,highlight_49,highlight_50,highlight_51,highlight_52,highlight_53,highlight_54,highlight_55,highlight_56,highlight_57,highlight_58,highlight_59,highlight_60,highlight_61,highlight_62,highlight_63,highlight_64,highlight_65,highlight_66,highlight_67,highlight_68,highlight_69,highlight_70,highlight_71,highlight_72,highlight_73,highlight_74,highlight_75,highlight_76,highlight_77,highlight_78,highlight_79,highlight_80,highlight_81,highlight_82,highlight_83,highlight_84,highlight_85,highlight_86,highlight_87,highlight_88,highlight_89,highlight_90,highlight_91,highlight_92,highlight_93,highlight_94,highlight_95,highlight_96,highlight_97,highlight_98,highlight_99,highlight_100,highlight_101,highlight_102,highlight_103,highlight_104
0,4476,23,34,1764,"North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),350,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,21211,23,34,74,"North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",300,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,

## Extracting time 

In [ ]:
string = ['8:30am – 10:30pm (Mon-Sun)', '12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat)', '8am – 11pm (Mon-Sun)', '11:30 AM to 11:30 PM', '12 Noon to 11:30 PM (Mon-Sun)']

In [ ]:
# pattern = "([0-9]+[:]*[0-9]*([AaPp][Mm])*) (([Tt][Oo])+[-]) ([0-9]+[:]*[0-9]*([AaPp][Mm])*)"
# pattern = '[0-9]+*[:]*[0-9]*[ ]*[AaPp][Mm]'
pattern = '[0-9]+[:]*[0-9]*[ ]*[a-zA-z]*'

In [ ]:
for s in string:
    x = re.findall(pattern, s)
    print(x)

['8:30am', '10:30pm']
['12:30PM', '12Midnight']
['8am', '11pm']
['11:30 AM', '11:30 PM']
['12 Noon', '11:30 PM']


In [ ]:
pattern = '[0-9]+[:]*[0-9]*[ ]*[a-zA-z]*'
def set_time(row):
    try:
        start_end = re.findall(pattern, row.timings)
    #     print(start_end)
        if(len(start_end) <= 1):
            start_end = ['0am', '12pm']
        row['start_time'] = start_end[0]
        row['end_time'] = start_end[1]
    except Exception as e:
        print("Error in:", row)
    return row

In [ ]:
df = df.apply(set_time, axis=1)

In [ ]:
df.head(5)

,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,highlights,aggregate_rating,delivery,cuisine_1,cuisine_2,cuisine_3,cuisine_4,cuisine_5,cuisine_6,cuisine_7,cuisine_8,cuisine_9,cuisine_10,cuisine_11,cuisine_12,cuisine_13,cuisine_14,cuisine_15,cuisine_16,cuisine_17,cuisine_18,cuisine_19,cuisine_20,cuisine_21,cuisine_22,cuisine_23,cuisine_24,cuisine_25,cuisine_26,cuisine_27,cuisine_28,cuisine_29,cuisine_30,cuisine_31,cuisine_32,cuisine_33,cuisine_34,cuisine_35,cuisine_36,cuisine_37,cuisine_38,cuisine_39,cuisine_40,cuisine_41,cuisine_42,cuisine_43,cuisine_44,cuisine_45,cuisine_46,cuisine_47,cuisine_48,cuisine_49,cuisine_50,cuisine_51,cuisine_52,cuisine_53,cuisine_54,cuisine_55,cuisine_56,cuisine_57,cuisine_58,cuisine_59,cuisine_60,cuisine_61,cuisine_62,cuisine_63,cuisine_64,cuisine_65,cuisine_66,cuisine_67,cuisine_68,cuisine_69,cuisine_70,cuisine_71,cuisine_72,cuisine_73,cuisine_74,cuisine_75,cuisine_76,cuisine_77,cuisine_78,cuisine_79,cuisine_80,cuisine_81,cuisine_82,cuisine_83,cuisine_84,cuisine_85,cuisine_86,cuisine_87,cuisine_88,cuisine_89,cuisine_90,cuisine_91,cuisine_92,cuisine_93,cuisine_94,cuisine_95,cuisine_96,cuisine_97,cuisine_98,cuisine_99,cuisine_100,cuisine_101,cuisine_102,cuisine_103,cuisine_104,cuisine_105,cuisine_106,cuisine_107,cuisine_108,cuisine_109,cuisine_110,cuisine_111,cuisine_112,cuisine_113,cuisine_114,cuisine_115,cuisine_116,cuisine_117,cuisine_118,cuisine_119,cuisine_120,cuisine_121,cuisine_122,cuisine_123,cuisine_124,cuisine_125,cuisine_126,cuisine_127,cuisine_128,cuisine_129,cuisine_130,cuisine_131,cuisine_132,cuisine_133,highlight_1,highlight_2,highlight_3,highlight_4,highlight_5,highlight_6,highlight_7,highlight_8,highlight_9,highlight_10,highlight_11,highlight_12,highlight_13,highlight_14,highlight_15,highlight_16,highlight_17,highlight_18,highlight_19,highlight_20,highlight_21,highlight_22,highlight_23,highlight_24,highlight_25,highlight_26,highlight_27,highlight_28,highlight_29,highlight_30,highlight_31,highlight_32,highlight_33,highlight_34,highlight_35,highlight_36,highlight_37,highlight_38,highlight_39,highlight_40,highlight_41,highlight_42,highlight_43,highlight_44,highlight_45,highlight_46,highlight_47,highlight_48,highlight_49,highlight_50,highlight_51,highlight_52,highlight_53,highlight_54,highlight_55,highlight_56,highlight_57,highlight_58,highlight_59,highlight_60,highlight_61,highlight_62,highlight_63,highlight_64,highlight_65,highlight_66,highlight_67,highlight_68,highlight_69,highlight_70,highlight_71,highlight_72,highlight_73,highlight_74,highlight_75,highlight_76,highlight_77,highlight_78,highlight_79,highlight_80,highlight_81,highlight_82,highlight_83,highlight_84,highlight_85,highlight_86,highlight_87,highlight_88,highlight_89,highlight_90,highlight_91,highlight_92,highlight_93,highlight_94,highlight_95,highlight_96,highlight_97,highlight_98,highlight_99,highlight_100,highlight_101,highlight_102,highlight_103,highlight_104,start_time,end_time
0,4476,23,34,1764,"North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),350,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8:30am,10:30pm
1,21211,23,34,74,"North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",300,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0

## Checkpoint

In [ ]:
df.to_csv('preprocess_v1.csv', encoding='UTF-8')

## load

In [ ]:
df = pd.read_csv('preprocess_v1.csv')

### converting time into 24 hr format

In [ ]:
def convert_time(row):
    try:
#         print("***********")
        start = ''.join([x for x in row['start_time'] if x.isnumeric()])
        if len(start) == 1:
            start = start.ljust(3, '0')
        elif len(start) == 2:
            start = start.ljust(4, '0')
        start = start.rjust(4, '0')
        if 'pm' in row['start_time'] or 'PM' in row['start_time'] or 'Pm' in row['start_time']:
            if '12' not in start:
                start = (str(int(start[:-2])+12) + start[-2:]).rjust(4, '0')
            else:
                start = (start[:-2] + start[-2:]).rjust(4, '0')
#         print(start)
        end = ''.join([x for x in row['end_time'] if x.isnumeric()])
#         print(end)
        if len(end) == 1:
            end = end.ljust(3, '0')
        elif len(end) == 2:
            end = end.ljust(4, '0')
        end = (str(int(end[:-2])+12) + end[-2:]).rjust(4, '0')
#         print(start, end)
        row['start'] = start
        row['end'] = end
    except Exception as e:
        print("Error in: ", row['start_time'], start, row['end_time'], end)
        print("Error: ", str(e))
    return row

In [ ]:
arr = [('8:30am', '10:30pm'), ('12:30PM', '12Midnight'), ('11am', '11pm')]
row = {}
for s, e in arr:
    row['start_time'] = s
    row['end_time'] = e
    convert_time(row)

In [ ]:
df = df.apply(convert_time, axis=1)

In [ ]:
df['start'] = df['start'].astype(int)
df['end'] = df['end'].astype(int)

In [ ]:
df.head(5)

,Unnamed: 0,name,establishment,city_id,locality_verbose,cuisines,timings,average_cost_for_two,highlights,aggregate_rating,delivery,cuisine_1,cuisine_2,cuisine_3,cuisine_4,cuisine_5,cuisine_6,cuisine_7,cuisine_8,cuisine_9,cuisine_10,cuisine_11,cuisine_12,cuisine_13,cuisine_14,cuisine_15,cuisine_16,cuisine_17,cuisine_18,cuisine_19,cuisine_20,cuisine_21,cuisine_22,cuisine_23,cuisine_24,cuisine_25,cuisine_26,cuisine_27,cuisine_28,cuisine_29,cuisine_30,cuisine_31,cuisine_32,cuisine_33,cuisine_34,cuisine_35,cuisine_36,cuisine_37,cuisine_38,cuisine_39,cuisine_40,cuisine_41,cuisine_42,cuisine_43,cuisine_44,cuisine_45,cuisine_46,cuisine_47,cuisine_48,cuisine_49,cuisine_50,cuisine_51,cuisine_52,cuisine_53,cuisine_54,cuisine_55,cuisine_56,cuisine_57,cuisine_58,cuisine_59,cuisine_60,cuisine_61,cuisine_62,cuisine_63,cuisine_64,cuisine_65,cuisine_66,cuisine_67,cuisine_68,cuisine_69,cuisine_70,cuisine_71,cuisine_72,cuisine_73,cuisine_74,cuisine_75,cuisine_76,cuisine_77,cuisine_78,cuisine_79,cuisine_80,cuisine_81,cuisine_82,cuisine_83,cuisine_84,cuisine_85,cuisine_86,cuisine_87,cuisine_88,cuisine_89,cuisine_90,cuisine_91,cuisine_92,cuisine_93,cuisine_94,cuisine_95,cuisine_96,cuisine_97,cuisine_98,cuisine_99,cuisine_100,cuisine_101,cuisine_102,cuisine_103,cuisine_104,cuisine_105,cuisine_106,cuisine_107,cuisine_108,cuisine_109,cuisine_110,cuisine_111,cuisine_112,cuisine_113,cuisine_114,cuisine_115,cuisine_116,cuisine_117,cuisine_118,cuisine_119,cuisine_120,cuisine_121,cuisine_122,cuisine_123,cuisine_124,cuisine_125,cuisine_126,cuisine_127,cuisine_128,cuisine_129,cuisine_130,cuisine_131,cuisine_132,cuisine_133,highlight_1,highlight_2,highlight_3,highlight_4,highlight_5,highlight_6,highlight_7,highlight_8,highlight_9,highlight_10,highlight_11,highlight_12,highlight_13,highlight_14,highlight_15,highlight_16,highlight_17,highlight_18,highlight_19,highlight_20,highlight_21,highlight_22,highlight_23,highlight_24,highlight_25,highlight_26,highlight_27,highlight_28,highlight_29,highlight_30,highlight_31,highlight_32,highlight_33,highlight_34,highlight_35,highlight_36,highlight_37,highlight_38,highlight_39,highlight_40,highlight_41,highlight_42,highlight_43,highlight_44,highlight_45,highlight_46,highlight_47,highlight_48,highlight_49,highlight_50,highlight_51,highlight_52,highlight_53,highlight_54,highlight_55,highlight_56,highlight_57,highlight_58,highlight_59,highlight_60,highlight_61,highlight_62,highlight_63,highlight_64,highlight_65,highlight_66,highlight_67,highlight_68,highlight_69,highlight_70,highlight_71,highlight_72,highlight_73,highlight_74,highlight_75,highlight_76,highlight_77,highlight_78,highlight_79,highlight_80,highlight_81,highlight_82,highlight_83,highlight_84,highlight_85,highlight_86,highlight_87,highlight_88,highlight_89,highlight_90,highlight_91,highlight_92,highlight_93,highlight_94,highlight_95,highlight_96,highlight_97,highlight_98,highlight_99,highlight_100,highlight_101,highlight_102,highlight_103,highlight_104,start_time,end_time,start,end
0,0,4476,23,34,1764,"North Indian, South Indian, Mithai, Street Foo...",8:30am – 10:30pm (Mon-Sun),350,"['Lunch', 'Takeaway Available', 'Credit Card',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8:30am,10:30pm,830,2230
1,1,21211,23,34,74,"North Indian, Mughlai, Rolls, Chinese, Fast Fo...","12:30PM to 12Midnight (Mon, Wed, Thu, Fri, Sat...",300,"['Delivery', 'No Alcohol Available', 'Dinner',...",4.4,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Checkpoint 2

In [ ]:
df.to_csv('preprocess_v2.csv', encoding='UTF-8')